In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from accelerate import init_empty_weights

model_name = "mosaicml/mpt-7b-instruct"
# model_name =  "stabilityai/stablelm-tuned-alpha-7b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config=config, trust_remote_code=True)

model.tie_weights()


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model


MPTForCausalLM(
  (transformer): MPTModel(
    (wte): Embedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

In [2]:
from accelerate import load_checkpoint_and_dispatch

model = load_checkpoint_and_dispatch(
    model,
    "/root/mpt-7b-instruct",
    device_map={},
    no_split_module_classes=["MPTBlock"],
)


In [5]:
def generate(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    input_ids = input_ids.to(0)
    output = model.generate(input_ids)
    return tokenizer.decode(output[0])


#### Loading mosaicml

In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
import torch

tokenizer = AutoTokenizer.from_pretrained("mosaicml/mpt-7b-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "mosaicml/mpt-7b-instruct", trust_remote_code=True,  torch_dtype=torch.bfloat16
)


/root/mosaic_ml/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/bd1748ec173f1c43e11f1973fc6e61cb3de0f327/attention.py:148: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `at

TypeError: to() received an invalid combination of arguments - got (str, torch_dtype=torch.dtype), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)


In [3]:
device = torch.device("cuda")
model.to(device, dtype=torch.bfloat16)

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): Embedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

In [48]:
from transformers import StoppingCriteria, StoppingCriteriaList

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)


# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

generate_kwargs = {
    "temperature": 0.1,
    "top_p": 1.0,
    "top_k": 0,
    # "max_new_tokens": 512,
    "use_cache": True,
    "do_sample": True,
    # "eos_token_id": tokenizer.eos_token_id,
    # "pad_token_id": tokenizer.pad_token_id,
    # "repetition_penalty": 1.1,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper

}


def generate(text):
    model.eval()
    text = f'{text["instruction"]}:\n{text["input"]}'
    s = PROMPT_FOR_GENERATION_FORMAT.format(instruction=text)
    print(s)
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"]
    input_ids = input_ids.to(device)
    stop = StopOnTokens()

    output = model.generate(
        input_ids,
        max_new_tokens=512,
        **generate_kwargs,
        stopping_criteria=StoppingCriteriaList([stop])
    )
    new_tokens = output[0, len(input_ids[0]) :]
    output_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return output_text

In [24]:
import json

with open("/app/data/data_alpaca.json", "r") as f:
    data = json.load(f)


In [25]:
# sample 20 examples from the data list
import random

random.seed(42)
data = random.sample(data, 20)

In [7]:
data[0]

{'instruction': 'Simplify the following piece of text in the Arabic language',
 'input': 'النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليم',
 'output': 'النظام الليمبي مهم للذاكرة والعواطف. يحتوي على جزءين رئيسيين: الحصبة الهيبوكامبية للذاكرة والأميجدالا للعواطف. يمكن أن تسبب مشاكل في النظام الليمبي صرعًا، وارتباكً

In [50]:
outputs = []
from tqdm import tqdm 

for d in tqdm(data):
    out = generate(d)
    d['preds'] = out
    outputs.append(d)

  0%|          | 0/20 [00:00<?, ?it/s]

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Simplify the following piece of text in the Arabic language:
النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليمبي: النظام الليم
### Response:



  5%|▌         | 1/20 [00:09<02:57,  9.32s/it]

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Simplify the following piece of text in the English language:
Channel Attention Module with Multi-scale Grid Average Pooling for Breast Cancer Segmentation in an Ultrasound Image.
Breast cancer accounts for the second-largest number of deaths in women around the world, and more than 8 percent of women will suffer from the disease in their lifetime. Mortality due to breast cancer can be reduced by its early and precise diagnosis. Many studies have investigated methods for segmentation, and computer-aided diagnosis based on deep learning techniques, in particular, have recently gained attention. However, recently proposed methods such as FCN, SegNet and U-Net still need to be further improved to provide better semantic segmentation when diagnosing breast cancer by ultrasound imaging, because of their low performance. In this paper, we propose a channel attention modu

 10%|█         | 2/20 [00:13<01:51,  6.20s/it]

...

 90%|█████████ | 18/20 [01:16<00:08,  4.04s/it]

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Simplify the following article conceptually in the English language:
نتائج عملية دلفي لجنة تطوير المناهج الدراسية للروبوتات الصدرية. مع انتشار اعتماد الإجراءات الروبوتية على نطاق واسع، يمكن توقع مخاطر إضافية مرتبطة بعرض التعلم. يذكر هذا المقال نتائج عملية دلفي لتحديد إجراءات لتعزيز تدريب جراحي الصدرية الروبوتي وتعزيز أداء آمن للتدخلات الروبوتية القائمة مثل سرطان الرئة و جراحة الورم الصدري.
### Response:



 95%|█████████▌| 19/20 [01:18<00:03,  3.35s/it]

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Simplify the following piece of text in the English language:
أسلوب النانوتوبات العصبية المتحركة المتحركة للأنبوب العصبي. يعتمد أداء الأسلوب النووي العصبي على الموازنة الحيوية والحساسية للمواجهة الالكترودية-الأنسجة. يتم تقييد الأسلوب العصبي الحالي من أداء كهربائي سيء بما في ذلك مقاومة أولية عالية وقدرة تخزين الشحن المنخفضة. بالإضافة إلى ذلك، فإنهما صلبان ميكانيكيا مما يسبب استجابة رد فعل خلية للأنبوب المزروع. في هذا التقرير، أظهرنا طريقة نموذجية جديدة لصنع أسلوب النانوتوبات العصبية المتحركة للغاية. يمكن تعديل هيكل الأسلوب النووي بدقة من خلال تغيير وقت التشديد الكتروني. الأداء الكهربائي من النانوتوبات البوليبروليتولية (PPY) والنانوتوبيتولين (POT4) (Polyothenol nanothenol) (POThenol) ، تمتحسين هذه الأسلوبات وتحسينه القدرة التشديد الكهربية من خلال عملية التشديد الكترونات البوليترونية البوليترونية والتشحن البيترونية. وقد تمت إظهار أن أسلوبات
### Response:



100%|██████████| 20/20 [01:23<00:00,  4.18s/it]


In [9]:
with open('/app/data/preds_mpt-7b-instruct.json', 'w') as f:
    json.dump(outputs, f, indent=4)

In [49]:
generate(data[1])

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
Simplify the following piece of text in the English language:
Channel Attention Module with Multi-scale Grid Average Pooling for Breast Cancer Segmentation in an Ultrasound Image.
Breast cancer accounts for the second-largest number of deaths in women around the world, and more than 8 percent of women will suffer from the disease in their lifetime. Mortality due to breast cancer can be reduced by its early and precise diagnosis. Many studies have investigated methods for segmentation, and computer-aided diagnosis based on deep learning techniques, in particular, have recently gained attention. However, recently proposed methods such as FCN, SegNet and U-Net still need to be further improved to provide better semantic segmentation when diagnosing breast cancer by ultrasound imaging, because of their low performance. In this paper, we propose a channel attention modu

'\nThe following is a simplified version of the above text:\nA new method for precisely segmenting breast cancer regions in ultrasound images is proposed. The method uses a channel attention module with multi-scale grid average pooling, which allows to use both global and local spatial information. The proposed network was constructed with an open source breast cancer ultrasound image dataset, and its performance was compared with those of other state-of-the-art deep-learning models for the segmentation of breast cancer. The experimental results showed that our network outperformed other segmentation methods, and the proposed channel attention module improved the performance of the network for breast cancer segmentation in ultrasound images.'

In [27]:
data[1]

{'instruction': 'Simplify the following piece of text in the English language',
 'input': 'Channel Attention Module with Multi-scale Grid Average Pooling for Breast Cancer Segmentation in an Ultrasound Image.\nBreast cancer accounts for the second-largest number of deaths in women around the world, and more than 8 percent of women will suffer from the disease in their lifetime. Mortality due to breast cancer can be reduced by its early and precise diagnosis. Many studies have investigated methods for segmentation, and computer-aided diagnosis based on deep learning techniques, in particular, have recently gained attention. However, recently proposed methods such as FCN, SegNet and U-Net still need to be further improved to provide better semantic segmentation when diagnosing breast cancer by ultrasound imaging, because of their low performance. In this paper, we propose a channel attention module with multiscale grid average pooling, for the precise segmentation of breast cancer region